In [1]:
import os
import sys
import time
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable
os.environ["PYSPARK_PYTHON"]        = sys.executable


In [2]:
from pyspark import SparkContext
from sympy import *
from drudge import *
from gristmill import *


In [3]:
import re

# ----------------------------------------------------------------------------
# 1) Start Spark & BCS quasiparticle drudge
# ----------------------------------------------------------------------------
ctx = SparkContext('local[2]', 'bcs_ccsd_z')
dr  = ReducedBCSDrudge(ctx)
dr.full_simplify = True
print("finished")

26/01/18 10:49:07 WARN Utils: Your hostname, Swarnamoys-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.87 instead (on interface en0)
26/01/18 10:49:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/18 10:49:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
                                                                                

finished


In [4]:
p,q,r,s,i,j,k,l = dr.names.A_dumms[:8]
u, v = IndexedBase('u'), IndexedBase('v')

In [5]:
P, Pdag, N = dr.names.P, dr.names.Pdag, dr.names.N
P_i_dag = (u[p]*v[p]+u[p]**2*Pdag[p]
     - u[p]*v[p]*N[p]
     - v[p]**2*P[p])

P_i_dag_i = (u[p+r]*v[p+r]+u[p+r]**2*Pdag[p+r]
     - u[p+r]*v[p+r]*N[p+r]
     - v[p+r]**2*P[p+r])

P_j_dag = (
        u[q]*v[q]
      + u[q]**2 * Pdag[q]
      - u[q]*v[q] * N[q]
      - v[q]**2 * P[q]
    )

P_k_dag = (
        u[r]*v[r]
      + u[r]**2 * Pdag[r]
      - u[r]*v[r] * N[r]
      - v[r]**2 * P[r]
    )
P_i = (
        (u[p])*(v[p])
      + (u[p])**2 * P[p]
      - (u[p])*(v[p]) * N[p]
      - (v[p])**2 * Pdag[p]
    )

P_i_i = (
        (u[p+r])*(v[p+r])
      + (u[p+r])**2 * P[p+r]
      - (u[p+r])*(v[p+r]) * N[p+r]
      - (v[p+r])**2 * Pdag[p+r]
    )

P_j = ((u[q])*(v[q])
      + (u[q])**2 * P[q]
      - (u[q])*(v[q]) * N[q]
      - (v[q])**2 * Pdag[q])

P_k = ((u[r])*(v[r])
      + (u[r])**2 * P[r]
      - (u[r])*(v[r]) * N[r]
      - (v[r])**2 * Pdag[r])
#hc = P_j_dag * P_i
N_i = 2*(v[p])*v[p] +((u[p])*u[p] - (v[p])*v[p])*N[p] +2*u[p]*(v[p])*Pdag[p]\
            +2*(u[p])*v[p]*P[p]
N_i_r = 2*(v[p+r])*v[p+r] +((u[p+r])*u[p+r] - (v[p+r])*v[p+r])*N[p+r] +2*u[p+r]*(v[p+r])*Pdag[p+r]\
            +2*(u[p+r])*v[p+r]*P[p+r]
N_j = 2*(v[q])*v[q] +((u[q])*u[q] - (v[q])*v[q])*N[q] +2*u[q]*(v[q])*Pdag[q]\
            +2*(u[q])*v[q]*P[q]

N_k = 2*(v[r])*v[r] +((u[r])*u[r] - (v[r])*v[r])*N[r] +2*u[r]*(v[r])*Pdag[r]\
       +2*(u[r])*v[r]*P[r]

In [ ]:
 
E0 = Symbol(r'G')
H120, H021 = IndexedBase('h120') , IndexedBase('h021')
H011, H110 = IndexedBase('h011'), IndexedBase('h110')
H100, H011 = IndexedBase('h100'), IndexedBase('h011')
H001, H020 = IndexedBase('h001'), IndexedBase('h020')
H010 = IndexedBase('h010') 
H000 = IndexedBase('h000')

#L = Range('L', 1, Lmax)
#dr.set_dumms(L, (p, q, r, s))
#dr.add_resolver_for_dumms()


In [ ]:
#Lag = dr.sum((p,L), E0*u[p]**2*v[p])
#Lag = 2*E0*u[p]**2*v[p]*v[q]*u[q]  + v[p]**2*u[p]*u[q] + 2*u[p]**2*v[p]*v[q]*u[q]
Lag =  2*u[p]**2*v[q]*v[r]
Lag = dr.einst(Lag)
#print(Lag.free_inds)   # free index *objects*
#print(Lag.free_vars)   # free scalar symbols


In [ ]:
Lag.display()

In [ ]:
dlag_dt = Lag.diff(v[p])
#Dlag_Dt = diff(Lag,u[p])
print('differentiation')

In [ ]:
dlag_dt.display()

In [6]:
H11, H20 , H02 = IndexedBase('H11'), IndexedBase('H20'), IndexedBase('H02')
H04, H40 = IndexedBase('H04'), IndexedBase('H40')
H22, Hb22 = IndexedBase('H22'), IndexedBase('HT22')
H31, H13 = IndexedBase('H31'), IndexedBase('H13')

In [7]:
expr = H11[p]*N[p] + H02[p]*Pdag[p] +H20[p]*P[p]+H22[p,q]*N[p]*N[q]+Hb22[p,q]*Pdag[p]*P[q]\
    +H40[p,q]*P[p]*P[q]+H04[p,q]*Pdag[p]*Pdag[q]+H13[p,q]*Pdag[p]*N[q]+H31[p,q]*N[p]*P[q]
H_N = dr.einst(expr).simplify().merge()

In [8]:
H_N.display()

<IPython.core.display.Math object>

In [9]:
t1, t2, t3,t4 = IndexedBase('t1'), IndexedBase('t2'), IndexedBase('t3'), IndexedBase('t4')

z1, z2 = IndexedBase('z1'), IndexedBase('z2')
z3, z4 = IndexedBase('z3'), IndexedBase('z4')

Z1 = dr.einst(z1[p]*P[p])
Z2 = dr.einst(Rational(1,2)*z2[p,q]*P[p]*P[q])
Z3 = dr.einst(Rational(1,6)*z3[p,q,r]*P[p]*P[q]*P[r])
Z4 = dr.einst(Rational(1,24)*z4[p,q,r,s]*P[p]*P[q]*P[r]*P[s])

T1 = dr.einst(t1[p] * Pdag[p])
T2 = dr.einst(Rational(1, 2) * t2[p,q] *Pdag[p]*Pdag[q])
T3 = dr.einst(Rational(1,6)*t3[p,q,r]*Pdag[p]*Pdag[q]*Pdag[r])
T4 = dr.einst(Rational(1,24)*t4[p,q,r,s]*Pdag[p]*Pdag[q]*Pdag[r]*Pdag[s])

Z =  Z1+Z2 +Z3+ Z4
cluster = T1+ T2 +T3+ T4
Z.display()
#cluster.display()

<IPython.core.display.Math object>

In [10]:
dr.set_symm(t2, Perm([1, 0],IDENT), valence=2) 
dr.set_symm(z2, Perm([1, 0],IDENT), valence=2) 
dr.set_symm(t3, Perm([1,0,2],IDENT),Perm([2,1,0],IDENT), Perm([0,2,1],IDENT), valence=3)
dr.set_symm(z3, Perm([1,0,2],IDENT),Perm([2,1,0],IDENT), Perm([0,2,1],IDENT), valence=3)
dr.set_symm(t4,Perm([1,0,2,3],IDENT),Perm([0,1,3,2],IDENT),Perm([0,2,1,3],IDENT))
dr.set_symm(z4,Perm([1,0,2,3],IDENT),Perm([0,1,3,2],IDENT),Perm([0,2,1,3],IDENT))
dr.set_symm(H04,Perm([1,0],IDENT))
dr.set_symm(H40,Perm([1,0],IDENT)) 
#dr.set_symm(t3, Perm([1,0,2],IDENT),Perm([2,1,0],IDENT), Perm([0,2,1],IDENT), valence=3)
print("Perm works!")

Perm works!


In [11]:
zero_term = [(H40[p,p],0),(H04[p,p],0),(t2[p,p],0),(t4[p,p,p,p],0),(t3[p,p,p],0),(z3[p,p,p],0),(z2[p,p],0),(z4[p,p,p,p],0)]

In [12]:
t0 = time.time()
Hbar = H_N
curr = H_N
fact = 1
for n in range(4):                              # up to 4-fold for 2-body H
    comm = (curr | cluster).simplify()          # ad_T^{n+1}(H_N)
    fact *= (n + 1)                             # 1!,2!,3!,4!
    Hbar += comm / fact                         # add 1/(n+1)! term
    curr = comm    
print("Hbar evaluation done",time.time()-t0)

[Stage 117:>                                                        (0 + 2) / 2]

Hbar evaluation done 146.54809093475342


In [13]:
E_eqn  = Hbar.simplify().eval_vev().simplify()
E_eqn = E_eqn.subst_all(zero_term)
#print(E_eqn.latex())
E_eqn.display()

<IPython.core.display.Math object>

In [ ]:
t0=time.time()
s1_eqn_CCSD = (P[p] * Hbar).simplify().eval_vev().simplify()
print('evaluation done',time.time()-t0)

In [ ]:
s1_eqn_CCSD = s1_eqn_CCSD.subst_all(zero_term)
print(s1_eqn_CCSD.latex())

In [ ]:
s2_eqn_CCSD = (P[p]*P[q] * Hbar).simplify().eval_vev().simplify()
s2_eqn_CCSD = s2_eqn_CCSD.subst_all(zero_term)
print('evaluation done')

In [ ]:
print(s2_eqn_CCSD.latex())

In [14]:
def drop_delta_terms_str(tens: Tensor) -> Tensor:
    # Remove any term whose printed form contains 'KroneckerDelta('
    filt = tens.terms.filter(lambda term: 'KroneckerDelta' not in str(term))
    return Tensor(tens._drudge, filt).merge().simplify()

s1_eqn_CCSD = drop_delta_terms_str(s1_eqn_CCSD)
s2_eqn_CCSD = drop_delta_terms_str(s2_eqn_CCSD)

NameError: name 's1_eqn_CCSD' is not defined

In [ ]:
print(s1_eqn_CCSD.latex())

In [ ]:
print(s2_eqn_CCSD.latex())

In [ ]:
'''Defining CC Lagrangian
L = <0|(1+Z) bar{H}|0> 
'''
'''Z1,Z2 = IndexedBase('z1'), IndexedBase('z2')
R1,R2 = IndexedBase('R1'), IndexedBase('R2')
 
expr1 = dr.define(R1[p],s1_eqn_CCSD)
 
expr2 = dr.define(R2[p,q],s2_eqn_CCSD)
L =  E_eqn + Z1[p]*s1_eqn_CCSD + Z2[p,q]*s2_eqn_CCSD
L = dr.einst(L).merge()'''

In [ ]:
print(L.latex())

In [ ]:
dL_dt_1 = L.diff(t1[p]).simplify().merge()

#dL_dt = L.diff(t1[r])


In [ ]:
print(dL_dt_1.latex())

In [ ]:
dL_dt_2 = L.diff(t2[r,s]).simplify().merge()

In [ ]:
print(dL_dt_2.latex())

In [ ]:
import time

t0 = time.time()
C = (Hbar | Pdag[p])
print("build C:", time.time()-t0)

t0 = time.time()
Cs = C.simplify()
print("C.simplify:", time.time()-t0)

t0 = time.time()
expr = (1+Z) * Cs
print("multiply:", time.time()-t0)

t0 = time.time()
expr_s = expr.simplify()
print("expr.simplify:", time.time()-t0)

t0 = time.time()
vev = expr_s.eval_vev()
print("eval_vev:", time.time()-t0)

t0 = time.time()
out = vev.simplify()
print("vev.simplify:", time.time()-t0)


In [ ]:
C = (Hbar | Pdag[p]).simplify()

term0 = C.eval_vev()

termZ = 0
for Zi in (Z1, Z2, Z3, Z4):
    termZ += (Zi*C).expand().eval_vev()

res = (term0 + termZ).simplify()


In [ ]:
'''Defining CC Lagrangian
L = <0|(1+Z) bar{H}|0> and dL/dt = 0 is the Z equations. dL/dt = <0|(1+Z)[bar{H},Q_{mu}]|0>
'''
t0 = time.time()
# 1) Build the commutator object
C = (Hbar | Pdag[p])            # if "|" is your commutator op

# 2) Simplify the commutator itself (usually shrinks a lot)
C = C.simplify()

# 3) Use linearity: <0|(1+Z) C|0> = <0|C|0> + <0|Z C|0>
term0 = C.eval_vev().simplify()

#termZ = (Z * C).simplify().eval_vev().simplify()   # crucial: eval_vev BEFORE simplify of the product
termZ = (Z * C).normal_order().eval_vev().simplify() 
dL_dt_1 = (term0 + termZ).simplify()
 
#dL_dt_1 = ((1+Z)*(Hbar|Pdag[p])).simplify().eval_vev().simplify()
print("finish time: ",time.time()-t0 )

[Stage 1143:=================================>                    (10 + 2) / 16]

In [16]:
dL_dt_1 = dL_dt_1.subst_all(zero_term)
print(dL_dt_1.latex())

\sum_{q \in A} z^{(1)}_{q}  HT^{(22)}_{q,p}   + \sum_{q \in A} 2 t^{(1)}_{q}  H^{(40)}_{q,p}   + \sum_{q \in A} 2 H^{(13)}_{q,p}  z^{(2)}_{q,p}  - \sum_{q \in A} 4 z^{(1)}_{p}  H^{(40)}_{q,p}  t^{(2)}_{q,p}  - \sum_{q \in A} 4 H^{(31)}_{p,p}  t^{(2)}_{q,p}  z^{(2)}_{q,p}  - \sum_{q \in A} 4 H^{(31)}_{q,p}  t^{(2)}_{q,p}  z^{(2)}_{q,p}  - \sum_{q \in A} 2 t^{(1)}_{q}^{2}  z^{(1)}_{q}  H^{(40)}_{q,p}  - \sum_{q \in A} 2 t^{(1)}_{q}^{2}  H^{(31)}_{p,q}  z^{(2)}_{q,p}  - \sum_{q \in A} 2 H^{(20)}_{p}  t^{(2)}_{q,p}  z^{(2)}_{q,p}  - \sum_{q \in A} 2 t^{(1)}_{p}  HT^{(22)}_{q,p}  z^{(2)}_{q,p}   + \sum_{q \in A} 2 t^{(1)}_{q}  z^{(1)}_{p}  H^{(31)}_{p,q}   + \sum_{q \in A} 2 t^{(1)}_{q}  z^{(1)}_{q}  H^{(31)}_{q,p}   + \sum_{q \in A} 4 t^{(1)}_{q}  H^{(22)}_{p,q}  z^{(2)}_{q,p}   + \sum_{q \in A} 4 t^{(1)}_{q}  H^{(22)}_{q,p}  z^{(2)}_{q,p}  - \sum_{q \in A} 4 t^{(1)}_{p}  t^{(1)}_{q}  z^{(1)}_{p}  H^{(40)}_{q,p}  - \sum_{q \in A} 4 t^{(1)}_{p}  t^{(1)}_{q}  H^{(31)}_{q,p}  z^{(2)}_{q,p}   

In [15]:
#dL_dt_2 = (1+Z)*(Hbar|Pdag[p]*Pdag[q]).simplify().eval_vev().simplify()
# 1) Build the commutator object
t0 = time.time()
C = (Hbar | (Pdag[p]*Pdag[q]))            # if "|" is your commutator op

# 2) Simplify the commutator itself (usually shrinks a lot)
C = C.simplify()

# 3) Use linearity: <0|(1+Z) C|0> = <0|C|0> + <0|Z C|0>
term0 = C.eval_vev().simplify()
termZ = (Z * C).simplify().eval_vev().simplify()   # crucial: eval_vev BEFORE simplify of the product

dL_dt_2 = (term0 + termZ).simplify()
print("finish", time.time()-t0)
dL_dt_2 = dL_dt_2.subst_all(zero_term)
print(dL_dt_2.latex())

finish 269.90134501457214
2 H^{(40)}_{p,q}   + H^{(20)}_{p}  z^{(1)}_{q}   + H^{(20)}_{q}  z^{(1)}_{p}   + 2 H^{(11)}_{p}  z^{(2)}_{p,q}   + 2 H^{(11)}_{q}  z^{(2)}_{p,q}   + 2 z^{(1)}_{p}  H^{(31)}_{p,q}   + 2 z^{(1)}_{q}  H^{(31)}_{q,p}   + 4 H^{(22)}_{p,p}  z^{(2)}_{p,q}   + 4 H^{(22)}_{p,q}  z^{(2)}_{p,q}   + 4 H^{(22)}_{q,p}  z^{(2)}_{p,q}   + 4 H^{(22)}_{q,q}  z^{(2)}_{p,q}  - 4 t^{(1)}_{p}  z^{(1)}_{p}  H^{(40)}_{p,q}  - 4 t^{(1)}_{p}  H^{(31)}_{p,p}  z^{(2)}_{p,q}  - 4 t^{(1)}_{p}  H^{(31)}_{q,p}  z^{(2)}_{p,q}  - 4 t^{(1)}_{q}  z^{(1)}_{q}  H^{(40)}_{p,q}  - 4 t^{(1)}_{q}  H^{(31)}_{p,q}  z^{(2)}_{p,q}  - 4 t^{(1)}_{q}  H^{(31)}_{q,q}  z^{(2)}_{p,q}  -  4 \delta_{p q} z^{(1)}_{p}  H^{(31)}_{p,p}  - 2 H^{(20)}_{p}  t^{(1)}_{p}  z^{(2)}_{p,q}  -  2 \delta_{p q} H^{(20)}_{p}  z^{(1)}_{p}  - 2 H^{(20)}_{q}  t^{(1)}_{q}  z^{(2)}_{p,q}   + 8 H^{(40)}_{p,q}  t^{(2)}_{p,q}  z^{(2)}_{p,q}   + 8 t^{(1)}_{p}  t^{(1)}_{q}  H^{(40)}_{p,q}  z^{(2)}_{p,q}   + \sum_{r \in A} HT^{(22)}_{r,p}  

In [ ]:
#dL_dt_3 = (1+Z)*(Hbar|Pdag[p]*Pdag[q]*Pdag[r]).simplify().eval_vev().simplify()
# 1) Build the commutator object
C = (Hbar | Pdag[p]*Pdag[q]*Pdag[r])            # if "|" is your commutator op

# 2) Simplify the commutator itself (usually shrinks a lot)
C = C.simplify()

# 3) Use linearity: <0|(1+Z) C|0> = <0|C|0> + <0|Z C|0>
term0 = C.eval_vev().simplify()
termZ = (Z * C).eval_vev().simplify()   # crucial: eval_vev BEFORE simplify of the product

dL_dt_3 = (term0 + termZ).simplify()

dL_dt_3 = dL_dt_3.subst_all(zero_term)
print(dL_dt_3.latex())
#print('finish')

In [ ]:
dL_dt_2 = dL_dt_2.subst_all(zero_term)
print(dL_dt_2.latex())

In [ ]:
dL_dt_2 = drop_delta_terms_str(dL_dt_2)

In [ ]:
L1 = IndexedBase('L1')
L2 = IndexedBase('L2')

expr1 = dr.define(L1[p],dL_dt_1)
 
print("LHS:", expr1.lhs)
print("RHS type:", type(expr1.rhs))
print("RHS:", expr1.rhs)
expr2 = dr.define(L2[p,q],dL_dt_2)
 

eval_equ0 = optimize(
    [expr1,expr2],
    interm_fmt='tau{}',drop_cutoff=2
)
print("Original cost:", get_flop_cost(eval_equ0))

fort = FortranPrinter()
print("Optimized cost:", get_flop_cost(eval_equ0))
 
evals = fort.doprint(eval_equ0)
with open('BCS_CCSD_Z.f90', 'w') as fp:
    fp.write(evals)

print("Printed!")

In [ ]:
Ex_H = ((1+Z)*(Hbar)).simplify().eval_vev().simplify()

In [ ]:
Ex_H = Ex_H.subst_all(zero_term)

In [ ]:
print(Ex_H.latex())

In [ ]:
E = IndexedBase('E')
 

expr1 = dr.define(E,Ex_H)
 
 
#Res2 = IndexedBase('Res2')
#expr2 = dr.define(L2[p,q],dL_dt_2)
 

eval_equ0 = optimize(
    [expr1],
    interm_fmt='tau{}',drop_cutoff=2
)
print("Original cost:", get_flop_cost(eval_equ0))

fort = FortranPrinter()
print("Optimized cost:", get_flop_cost(eval_equ0))
 
evals = fort.doprint(eval_equ0)
with open('BCS_CCSD_Z_Energy.f90', 'w') as fp:
    fp.write(evals)

print("Printed!")

In [ ]:
#Lmax = symbols('Lmax', integer=True, positive=True)
#L = Range('L', 1, Lmax)  # symbolic chain 1..Lmax (boundary handling is symbolic)
#dr.set_dumms(L, (p,q))
#expr = Rational(1,4)*(Pdag[p] + P[p]) - Rational(1,2)*(N[p-1]-1)*(Pdag[p] + P[p])*(N[p+1] - 1) +Rational(1,4)*G*(N[p-1] - 1)*(N[p+1] - 1)
#expr =  Rational(1,4)*(N[p]*N[p+r] - N[p] - N[p+r] +1)
expr =  Rational(1,4)*(N[p]*N[q] - N[p] - N[q] +1) + (Pdag[p]*P[q] + Pdag[q]*P[p])/2
#expr =  Rational(1,4)*(N_i*N_i_r - N_i - N_i_r +1)
#expr = Rational(1,4)*(N_i*N_j - N_i - N_j +1)
#expr = dr.sum((p,L),(q,L),expr).merge()
#expr = dr.sum((p,L),expr).merge()
expr = dr.einst(expr).normal_order().simplify().merge()
expr.display()

In [17]:
#Sz_pSz_q = Rational(1,4)*(N_i*N_i_r - N_i - N_i_r +1)
Sz_pSz_q = Rational(1,4)*(N_i*N_j - N_i - N_j +1)
Sz_pSz_q = dr.einst(Sz_pSz_q).normal_order().simplify().merge()
Sz_pSz_q.display()

<IPython.core.display.Math object>

In [ ]:
Sz2.display()

In [18]:
t0 =time.time()
Sz_pSz_qbar = Sz_pSz_q
curr = Sz_pSz_q
fact = 1
for n in range(4):                              # up to 4-fold for 2-body H
    comm = (curr | cluster).simplify()          # ad_T^{n+1}(H_N)
    fact *= (n + 1)                             # 1!,2!,3!,4!
    Sz_pSz_qbar += comm / fact                         # add 1/(n+1)! term
    curr = comm    
print("Sz_pSz_qbar_2 evaluation done ", time.time()-t0)

[Stage 2099:>                                                       (0 + 2) / 2]

Sz_pSz_qbar_2 evaluation done  13.811956882476807


In [19]:
t0 = time.time()
one = IndexedBase("one")   # you will set one(i)=1 in code
Ex_Sz_pSz_qbar = one[p]*one[q]*((1+Z)*(Sz_pSz_qbar)).simplify().eval_vev().simplify()
print("finished ", time.time()-t0)

finished  284.2462410926819


In [20]:
Ex_Sz_pSz_qbar  = Ex_Sz_pSz_qbar.subst_all(zero_term)
 
Ex_Sz_pSz_qbar = drop_delta_terms_str(Ex_Sz_pSz_qbar)


print(Ex_Sz_pSz_qbar.latex())

- \sum_{r \in A} \sum_{s \in A} \sum_{p_{0} \in A} u_{r}^{2}  v_{r}^{2}  one_{p}  one_{q}  t^{(2)}_{r,p_{0}}  t^{(2)}_{r,s}  z^{(2)}_{s,p_{0}}  - \sum_{r \in A} \sum_{s \in A} \sum_{p_{0} \in A} 2 v_{r}^{2}  one_{p}  one_{q}  t^{(1)}_{r}  u_{s}  v_{s}  t^{(2)}_{s,p_{0}}  z^{(2)}_{r,p_{0}}  - \sum_{r \in A} \sum_{s \in A} \sum_{p_{0} \in A} 2 v_{s}^{2}  one_{p}  one_{q}  t^{(1)}_{r}  u_{r}  v_{r}  t^{(2)}_{s,p_{0}}  z^{(2)}_{s,p_{0}}   + \sum_{r \in A} \sum_{s \in A} \sum_{p_{0} \in A} 2 u_{r}^{2}  one_{p}  one_{q}  t^{(1)}_{r}  u_{s}  v_{s}  t^{(2)}_{s,p_{0}}  z^{(2)}_{r,p_{0}}   + \sum_{r \in A} \sum_{s \in A} \sum_{p_{0} \in A} 2 u_{s}^{2}  one_{p}  one_{q}  t^{(1)}_{r}  u_{r}  v_{r}  t^{(2)}_{s,p_{0}}  z^{(2)}_{s,p_{0}}   + \sum_{r \in A} \sum_{s \in A} \sum_{p_{0} \in A} 2 one_{p}  one_{q}  u_{p_{0}}  u_{r}  v_{p_{0}}  v_{r}  t^{(2)}_{r,s}  z^{(2)}_{s,p_{0}}  - \sum_{r \in A} \sum_{s \in A} \sum_{p_{0} \in A} 4 one_{p}  one_{q}  u_{r}  u_{s}  v_{r}  v_{s}  t^{(2)}_{r,p_{0}}  t^{(2)

In [ ]:
s1_eqn_CCSD = drop_delta_terms_str(s1_eqn_CCSD)
s2_eqn_CCSD = drop_delta_terms_str(s2_eqn_CCSD)

In [23]:
SzSz = IndexedBase('SzSz')
 
expr1 = dr.define(SzSz[p,q],Ex_Sz_pSz_qbar)
 
#expr1 = dr.define(L1[p],dL_dt_1)
 
 
#Res2 = IndexedBase('Res2')
#expr2 = dr.define(L2[p,q],dL_dt_2)

print("LHS:", expr1.lhs)
print("RHS type:", type(expr1.rhs))
print("RHS:", expr1.rhs)
 
'''eval_equ0 = optimize(
    [expr1],
    interm_fmt='tau{}',drop_cutoff=2
)'''
stats = {}
t0 = time.time()
eval_equ0 = optimize(
    [expr1],
    interm_fmt="tau{}",
    drop_cutoff=2,
    remove_shallow=False,
    opt_sum=False,
    simplify=True,
    opt_symm=True,
    stats=stats,
)

 
 

 
print(stats)
#print("Original cost:", get_flop_cost(eval_equ0))

fort = FortranPrinter()
print("Optimized cost:", get_flop_cost(eval_equ0))
 
evals = fort.doprint(eval_equ0)
with open('BCS_CCSD_SzSz.f90', 'w') as fp:
    fp.write(evals)

print("Printed!", time.time()-t0)

LHS: SzSz[p, q]
RHS type: <class 'drudge.drudge.TensorDef'>
RHS: SzSz[p, q] = sum_{r, s, p0} -one[p]*one[q]*t2[r, p0]*t2[r, s]*u[r]**2*v[r]**2*z2[s, p0]
 + sum_{r, s, p0} -2*one[p]*one[q]*t1[r]*t2[s, p0]*u[s]*v[r]**2*v[s]*z2[r, p0]
 + sum_{r, s, p0} -2*one[p]*one[q]*t1[r]*t2[s, p0]*u[r]*v[r]*v[s]**2*z2[s, p0]
 + sum_{r, s, p0} 2*one[p]*one[q]*t1[r]*t2[s, p0]*u[r]**2*u[s]*v[s]*z2[r, p0]
 + sum_{r, s, p0} 2*one[p]*one[q]*t1[r]*t2[s, p0]*u[r]*u[s]**2*v[r]*z2[s, p0]
 + sum_{r, s, p0} 2*one[p]*one[q]*t2[r, s]*u[p0]*u[r]*v[p0]*v[r]*z2[s, p0]
 + sum_{r, s, p0} -4*one[p]*one[q]*t2[r, p0]*t2[r, s]*u[r]*u[s]*v[r]*v[s]*z2[r, p0]
 + sum_{r, s, p0} -2*one[p]*one[q]*t1[r]**2*t2[s, p0]*u[r]*u[s]*v[r]*v[s]*z2[r, p0]
 + sum_{r, s, p0} 2*one[p]*one[q]*t1[r]*t2[s, p0]*u[r]*u[s]*v[r]*v[s]*z1[p0]
 + sum_{r, s, p0} -4*one[p]*one[q]*t1[r]*t1[s]*t2[r, p0]*u[r]*u[s]*v[r]*v[s]*z2[r, p0]
 + one[p]*one[q]/4
 + -one[p]*one[q]*v[p]**2/2
 + -one[p]*one[q]*v[q]**2/2
 + one[p]*one[q]*v[p]**2*v[q]**2
 + sum_{r} one[p]*

In [32]:
#SpSq = Rational(1,4)*(N[p]*N[q] - N[p] - N[q] +1) + Rational(1,2)*((Pdag[p]*P[q] + Pdag[q]*P[p]))
SpSq = Rational(1,4)*(N_i*N_j - N_i - N_j +1) + Rational(1,2)*((P_i_dag*P_j + P_j_dag*P_i))
SpSq = dr.einst(SpSq).normal_order().simplify().merge()
SpSq.display()

<IPython.core.display.Math object>

In [33]:
t0 =time.time()
SpSqbar = SpSq
curr = SpSq
fact = 1
for n in range(4):                              # up to 4-fold for 2-body H
    comm = (curr | cluster).simplify()          # ad_T^{n+1}(H_N)
    fact *= (n + 1)                             # 1!,2!,3!,4!
    SpSqbar += comm / fact                         # add 1/(n+1)! term
    curr = comm    
print("SpSqbar_2 evaluation done ", time.time()-t0)

[Stage 5689:>                                                       (0 + 2) / 2]

SpSqbar_2 evaluation done  25.833006143569946


In [37]:
t0 = time.time()
one = IndexedBase("one")   # you will set one(i)=1 in code
Ex_SpSqbar = one[p]*one[q]*((1+Z)*(SpSqbar)).simplify().eval_vev().simplify()
print("finished ", time.time()-t0)

finished  626.7302529811859


In [39]:
SpSq = IndexedBase('SpSq')
 
expr1 = dr.define(SpSq[p,q],Ex_SpSqbar)
 
#expr1 = dr.define(L1[p],dL_dt_1)
 
 
#Res2 = IndexedBase('Res2')
#expr2 = dr.define(L2[p,q],dL_dt_2)

print("LHS:", expr1.lhs)
print("RHS type:", type(expr1.rhs))
print("RHS:", expr1.rhs)
 
'''eval_equ0 = optimize(
    [expr1],
    interm_fmt='tau{}',drop_cutoff=2
)'''
stats = {}
t0 = time.time()
eval_equ0 = optimize(
    [expr1],
    interm_fmt="tau{}",
    drop_cutoff=2,
    remove_shallow=False,
    opt_sum=False,
    simplify=True,
    opt_symm=True,
    stats=stats,
)

 
 

 
print(stats)
#print("Original cost:", get_flop_cost(eval_equ0))

fort = FortranPrinter()
print("Optimized cost:", get_flop_cost(eval_equ0))
 
evals = fort.doprint(eval_equ0)
with open('BCS_CCSD_SpSq.f90', 'w') as fp:
    fp.write(evals)

print("Printed!", time.time()-t0)

LHS: SpSq[p, q]
RHS type: <class 'drudge.drudge.TensorDef'>
RHS: SpSq[p, q] = sum_{r} one[p]*one[q]*v[r]**4
 + sum_{r} one[p]*one[q]*u[r]**2*v[r]**2
 + sum_{r} one[p]*one[q]*t2[r, r]*u[r]**2*z2[r, r]
 + sum_{r} one[p]*one[q]*t1[r]*u[r]**3*v[r]
 + sum_{r} one[p]*one[q]*u[r]**3*v[r]*z1[r]
 + sum_{r} one[p]*one[q]*t1[r]*u[r]**4*z1[r]
 + sum_{r} one[p]*one[q]*t1[r]*v[r]**2*z1[r]
 + sum_{r} one[p]*one[q]*t1[r]*u[r]*v[r]**3
 + sum_{r} one[p]*one[q]*u[r]*v[r]**3*z1[r]
 + sum_{r} one[p]*one[q]*t2[r, r]*v[r]**4*z2[r, r]
 + sum_{r} -one[p]*one[q]*t1[r]*u[r]**2*z1[r]
 + sum_{r} -one[p]*one[q]*t2[r, r]*u[r]**4*z2[r, r]
 + sum_{r} -one[p]*one[q]*t2[r, r]*v[r]**2*z2[r, r]
 + sum_{r} -one[p]*one[q]*t1[r]*v[r]**4*z1[r]
 + sum_{r} -one[p]*one[q]*t1[r]*u[r]*v[r]
 + sum_{r} -one[p]*one[q]*u[r]*v[r]*z1[r]
 + sum_{r} one[p]*one[q]*t1[r]**2*u[r]*v[r]*z1[r]
 + sum_{r} one[p]*one[q]*t1[r]*u[r]**2*v[p]**2*z1[r]
 + sum_{r} one[p]*one[q]*t1[r]*u[r]**2*v[q]**2*z1[r]
 + sum_{r} one[p]*one[q]*t2[r, r]*v[p]**2*v[r]*

{'Number of nodes': 2047}
Optimized cost: 449*na**2 + 557*na + 48
Printed! 82.44714379310608
